## 基于Densenet&Xception融合的iFLYTEK广告图片素材分类算法挑战赛baseline


这个融合模型是我在打比赛的时候随手搞出来的，一个开了脑洞的版本。


这个数据集实际上的难点就是长尾数据以及细粒度分类问题。


解题思路就是找一个足够强悍的模型，再加上飞桨的预训练模型，耐心调参，就会有一个不错的名次。


对于大多数分类问题，我比较习惯使用的是ResNet，DenseNet。


对于这种比较简单的图像分类任务，首先呢，大家可以尝试一下各种预训练模型，


简单的跑一下，对比一下和第一名的差距，如果还是没有太好的思路，那么就可以尝试本项目的思路。


大家如有任何问题，欢迎在评论区交流。[码字不易，大家关注我一下吧](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/228777)





关于比赛的详细介绍，请大家关注我另一个项目：[iFLYTEK广告图片素材分类算法挑战赛解题思路及baseline](https://aistudio.baidu.com/aistudio/projectdetail/2113367)

### 数据解压

In [ ]:
!cd 'data/data98715' && unzip -q fusion.zip
!cd 'data/data98606' && unzip -q data_advertising.zip

### 数据预处理

In [ ]:
import os
import json
import cv2

rootDir = 'data/data98606/data_advertising/train2'

list = []

image_id = []
category_id = []

def Test1(rootDir):
    list_dirs = os.walk(rootDir)
   

    img_id = 0
    for root, dirs, files in list_dirs:
        for d in dirs:
            # print(os.path.join(root, d))
            path = os.path.join(root, d)
            cat_id = int(d)
            for im in os.listdir(path):
                dict = {}
                img = cv2.imread(os.path.join(path, im))
                # print(os.path.join(path, im))
                img_h = img.shape[0]
                img_w = img.shape[1]
                dict['image_id'] = img_id
                img_id += 1
                dict['fpath'] = os.path.join(path, im)
                dict['im_height'] = img_h
                dict['im_width'] = img_w
                dict['category_id'] = cat_id
                list.append(dict)
                image_id.append(d+'/'+im)
                category_id.append(cat_id)

Test1(rootDir)

import pandas as pd

img = pd.DataFrame(image_id)
img = img.rename(columns = {0:"image_id"})
img['category_id'] = category_id

img.to_csv('work/train2.csv', index=False)

### 模型介绍


为什么要选择这两个模型呢？


Inception结构的网络和ResNet结构的网络的分别代表着深度神经网络的两个方向，


一是网络的宽度，二是网络的深度，神经网络变宽和变深已经被证明是可行的。


所以，本项目选择了两个非常具有代表性的两个网络进行融合。


1. Xception
2. DenseNet


这两个网络分别关注于网络的深度以及宽度，将这两个模型融合起来，共同提取图像的特征。

### Xception




先进行普通卷积操作，再对 1×1 1×11×1 卷积后的每个channel分别进行 3×3 3×33×3 卷积操作，最后将结果 concat：


![](https://img-blog.csdnimg.cn/20181206103219193.png)


***传统卷积的实现过程：***


![](https://img-blog.csdnimg.cn/20181206105510344.png)


***Depthwise Separable Convolution的实现过程：***


![](https://img-blog.csdnimg.cn/20181206105614489.png)





### DenseNet


相比ResNet，DenseNet提出了一个更激进的密集连接机制：即互相连接所有的层，具体来说就是每个层都会接受其前面所有层作为其额外的输入。


#### 结构对比(CNN, ResNet, DenseNet)：
  
![](https://ai-studio-static-online.cdn.bcebos.com/dd5a5d175bf14ad59f750a326d8d9c11bd2672fa1c7e41a6a4ac27ef159773ce)
  
  
#### DenseNet结构图

![](https://ai-studio-static-online.cdn.bcebos.com/d5c08fe7673b40b5ba2423fc0ea4f1a58238385c48404b6faa04652fbbc7468a)

  
  
  

DenseNet核心思想在于建立了不同层之间的连接关系，充分利用了feature，进一步减轻了梯度消失问题，加深网络不是问题，而且训练效果非常好。


## 模型融合&训练


这里只做简单的融合，虽然这种融合方式一点也不优雅，但是确实能涨分。


模型融合方式如下图所示，分别由两个模型提取特征，然后做一个拼接，通过全连接层输出预测概率。


大家需要注意，本项目中只用了一个全连接层，图中的两个全连接层的结构大家可以自己尝试。

### 融合模型的结构




![](https://ai-studio-static-online.cdn.bcebos.com/a53c55eccc414bc0977478785244d47d7baa87f134f146f190fcd447781496f1)


In [ ]:
print('*************begin**************')
print("+++++++++++++++++++++++++++++++")

In [ ]:
# !cd 'data/data98715' && unzip -d /home/aistudio/data fusion.zip
!cd 'data/data98606' && unzip -d /home/aistudio/data data_advertising.zip

In [ ]:
print('*************done1111111111111111**************')

In [ ]:
print('*************begin train**************')

In [ ]:
!cd 'work' && rm -r __pycache__/

In [ ]:
!cd 'work' && python train_dense.py

In [ ]:
!cd 'work/models/dense_model' && rm -r fc_0.b_0
!cd 'work/models/dense_model' && rm -r fc_0.w_0

In [ ]:
!cd 'work' && rm -r __pycache__/

In [ ]:
!cd 'work' && python train_xception.py

In [ ]:
!cd 'work/models/dense_model' && rm -r fc_0.b_0
!cd 'work/models/dense_model' && rm -r fc_0.w_0

In [ ]:
print('*************begin train_all**************')

In [ ]:
!cd 'work' && rm -r __pycache__/

In [ ]:
!cd 'work' && python train_all.py

In [ ]:
print('*************begin train_fine**************')

In [ ]:
!cd 'work' && rm -r __pycache__/

In [ ]:
!cd 'work' && python train_all_finetune.py

In [ ]:
!cd 'work' && python predict.py

In [ ]:
print('*************ALL DONE**************')

## 总结


总体来说，这种融合方法不是很优雅，相对于计算量的提升所带来的精度提升收益不是很大，


但是，这个比赛只要求精度，并且还有AIstudio的免费算力，大家可以尽情的跑一些大模型。



下面是我的得分，大家可以调整迭代次数、学习率、图像尺寸等参数，或者增加全连接层，添加DropOut，来调整网络，大家加油哦。


![](https://ai-studio-static-online.cdn.bcebos.com/f2cb47dc054b4ac0ab2eb562d65ec10211e5121160c444ea900619d2532ed220)


大家如果有任何问题欢迎Fork交流哦。[来AI Studio互粉吧~等你哦~ ](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/228777)